# DEVELOPING THE MAZE GENERATOR

## PREPARING THE DEVELOPMENT ENVIRONMENT

Importing the libraries:

In [1]:
from collections import deque
from numpy.random import shuffle, \
                         randint

Defining the auxiliary class with the disjoint set data structure:

- **n** is the number of supernodes
- **v** represents the parents (initially each disjoint set has a single vertex)

> **Note**: The compression technique used in the find method of disjoint set data structures is aimed at optimizing the process of finding the root or representative element of a set. In this technique, called path compression, as the method traverses the tree-like structure of the disjoint sets to find the root element, it also updates the parent pointers along the path to directly point to the root. This means that subsequent calls to find for the same element will take shorter paths, improving the efficiency of the operation.

In [2]:
class UnionFind:
    def __init__(self, n):
        self.n = n
        self.v = list(range(n))

    def find(self, u):
        while u != self.v[u]:
            self.v[u] = self.v[self.v[u]]
            u = self.v[u]

        return u

    def union(self, u, v):
        root_u, root_v = self.find(u), self.find(v)

        if root_u == root_v:
            return False
        else:
            self.v[root_v] = root_u
            self.n -= 1

            return True

Creating an auxiliary function to print the maze:

In [3]:
def is_conn(graph, v, w):
    return v in graph[w]

def get_row(maze, r, c, n):
    v = n * r + c

    return ' ' if is_conn(maze, v, v+1) else '|'

def get_col(maze, r, c, n):
    v = n * r + c

    return '  ' if is_conn(maze, v, v+n) else '_ '

def print_maze(n, maze):
    for r in range(n):
        for c in range(n-1):
            print('0', end=get_row(maze, r, c, n))

        print('0')

        if r != n-1:
            for c in range(n):
                print(end=get_col(maze, r, c, n))

            print()

## DFS GENERATOR

The randomized depth-first search algorithm is utilized to generate mazes by exploring the grid space randomly, prioritizing unvisited cells at each step:

- Its recursive nature allows efficient backtracking when no unvisited cells are available, ensuring thorough exploration
- By introducing randomness into the direction selection process, the algorithm generates unique and intricate mazes

In [4]:
class DFSMazeGenerator:
    def __init__(self, n=100):
        self.n = n
        self.shape = int(n**0.5)

        self.edges = self.generate_edges()

    def moves(self, v):
        moves = []

        if v % self.shape:
            moves.append(v - 1)
        if (v + 1) % self.shape:
            moves.append(v + 1)
        if v >= self.shape:
            moves.append(v - self.shape)
        if v < self.n - self.shape:
            moves.append(v + self.shape)

        return moves

    def generate_edges(self):
        return [self.moves(v)
                for v in range(self.n)]

    def shuffle_edges(self):
        edges = self.generate_edges()
        for adj in edges:
            shuffle(adj)

        return edges

    def generate_maze(self):
        maze = [[] for _ in range(self.n)]

        stack  = deque([])
        explored = set([])
        frontier = self.shuffle_edges()

        v = 0
        explored.add(v)
        while True:
            while not frontier[v]:
                if not stack:
                    return maze

                v = stack.pop()

            w = frontier[v].pop()

            if w not in explored:
                maze[v].append(w)
                maze[w].append(v)

                stack.append(v)
                explored.add(w)
                v = w

Checking the possible valid edges:

In [5]:
generator = DFSMazeGenerator()

print_maze(generator.shape, generator.edges)

0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0


Testing the generator and printing the results:

In [6]:
maze = generator.generate_maze()

print_maze(generator.shape, maze)

0 0|0 0 0 0|0 0 0 0
_     _ _   _   _   
0|0 0|0 0|0|0 0|0|0
  _ _         _     
0 0 0 0|0|0 0|0|0 0
  _ _ _ _ _ _     _ 
0|0 0 0 0 0 0|0 0|0
    _ _   _     _   
0 0|0 0 0|0 0|0 0|0
  _   _ _   _ _     
0|0|0 0 0|0 0 0|0|0
    _ _   _ _ _     
0 0|0 0|0 0 0 0 0|0
_     _ _ _ _ _ _   
0 0|0 0 0 0 0|0 0 0
  _ _ _   _     _   
0|0 0 0|0 0|0|0 0|0
    _   _     _ _   
0 0|0 0 0 0|0 0 0 0


Checking the efficiency of the maze generator:

In [7]:
%%timeit

_ = generator.generate_maze()

368 µs ± 102 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## PRIM GENERATOR

The prim's algorithm is often utilized in creating mazes due to its efficiency and simplicity:

- Initially, it selects point 0 within the maze space as the starting point
- Then, it incrementally expands the maze by adding paths, randomly selecting available edges from the set of unexplored cells
- This process continues until all cells are connected, forming a complete maze without cycles

> **Note**: Ultimately, the resulting maze possesses a unique and balanced structure, ideal for applications in games, simulations, or route optimization problems.

In [8]:
class PrimMazeGenerator:
    def __init__(self, n=100):
        self.n = n
        self.shape = int(n**0.5)

        self.edges = self.generate_edges()

    def moves(self, v):
        moves = []

        if v % self.shape:
            moves.append(v - 1)
        if (v + 1) % self.shape:
            moves.append(v + 1)
        if v >= self.shape:
            moves.append(v - self.shape)
        if v < self.n - self.shape:
            moves.append(v + self.shape)

        return moves

    def generate_edges(self):
        return [self.moves(v)
                for v in range(self.n)]

    def generate_maze(self):
        maze = [[] for _ in range(self.n)]

        V = [True ] + \
            [False] * (self.n - 1)
        E = [(0, w) for w in self.edges[0]]

        n = 1
        while n < self.n:
            v, w = E.pop(randint(0, len(E)))

            if not V[w]:
                maze[v].append(w)
                maze[w].append(v)

                V[w] = True
                for w2 in self.edges[w]:
                    if not V[w2]:
                        E.append((w, w2))

                n += 1

        return maze

Checking the possible valid edges:

In [9]:
generator = PrimMazeGenerator()

print_maze(generator.shape, generator.edges)

0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0


Testing the generator and printing the results:

In [10]:
maze = generator.generate_maze()

print_maze(generator.shape, maze)

0 0 0 0 0 0 0 0|0 0
    _             _ 
0|0 0|0|0|0|0|0 0|0
  _   _       _     
0 0|0 0|0|0|0|0 0 0
    _   _       _   
0|0 0|0|0|0|0|0|0 0
  _       _     _   
0 0|0|0 0|0 0|0 0|0
    _ _ _     _ _ _ 
0|0 0 0 0|0|0 0 0 0
  _ _ _ _ _   _     
0|0|0 0 0|0|0 0|0|0
      _ _   _   _ _ 
0 0 0|0 0 0 0 0 0 0
_   _ _ _           
0|0|0|0|0 0|0|0|0|0
        _     _   _ 
0 0 0 0 0|0|0|0 0 0


Checking the efficiency of the maze generator:

In [11]:
%%timeit

_ = generator.generate_maze()

1.53 ms ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## KRUSKAL GENERATOR

Defining the class that implements a maze generator inspired by a randomized kruskal algorithm. The maze is represented using an adjacency list, as the resulting graph is expected to be sparse.

In [12]:
class KruskalMazeGenerator:
    def __init__(self, n=100):
        self.n = n
        self.shape = int(n**0.5)

        self.edges = self.generate_edges()

    def moves(self, v):
        moves = []

        if (v + 1) % self.shape:
            moves.append((v, v + 1))
        if v + self.shape < self.n:
            moves.append((v, v + self.shape))

        return moves

    def generate_edges(self):
        return [edge
                for v in range(self.n)
                for edge in self.moves(v)]

    def generate_maze(self):
        forest = UnionFind(self.n)
        maze   = [[] for _ in range(self.n)]

        shuffle(self.edges)

        for v, w in self.edges:
            if forest.union(v, w):
                maze[v].append(w)
                maze[w].append(v)

                if forest.n == 1:
                    return maze

Checking the possible valid edges:

In [13]:
generator = KruskalMazeGenerator()

edges = [[] for _ in range(generator.n)]
for v, w in generator.edges:
    edges[v].append(w)
    edges[w].append(v)

print_maze(generator.shape, edges)

0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0
                    
0 0 0 0 0 0 0 0 0 0


Testing the generator and printing the results:

In [14]:
maze = generator.generate_maze()

print_maze(generator.shape, maze)

0 0 0|0|0 0 0 0 0|0
  _       _ _ _     
0|0|0|0 0 0 0 0|0 0
        _ _ _   _   
0 0|0 0 0 0|0|0 0|0
        _     _ _ _ 
0|0|0|0 0|0 0|0|0 0
_ _ _   _   _   _   
0 0|0|0 0|0|0|0 0 0
  _   _   _       _ 
0 0|0|0|0 0 0|0|0 0
_       _ _     _ _ 
0|0 0 0 0|0 0 0 0|0
        _           
0 0|0|0|0 0|0|0|0 0
  _ _     _     _   
0|0|0|0 0 0|0|0|0|0
          _   _     
0|0 0 0|0 0|0 0|0 0


Checking the efficiency of the maze generator:

In [15]:
%%timeit

_ = generator.generate_maze()

236 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
